# 📊 Customer Clustering - Model Evaluation

This notebook provides comprehensive evaluation and comparison of all trained clustering models.

## Evaluation Sections:
1. **Load Models & Results** - Import all trained models
2. **Performance Metrics** - Detailed metric comparison
3. **Cluster Visualization (2D/3D)** - PCA and t-SNE projections
4. **Cluster Profiles** - Analyze characteristics of each cluster
5. **Silhouette Analysis** - Per-cluster silhouette scores
6. **Cluster Size Distribution** - Balance analysis
7. **Feature Importance** - Which features drive clustering?
8. **Business Insights** - Actionable customer segments

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
from matplotlib.patches import Patch

# Clustering and metrics
from sklearn.metrics import silhouette_score, silhouette_samples, calinski_harabasz_score, davies_bouldin_score

# Dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Statistics
from scipy.spatial.distance import cdist

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ All libraries imported successfully!")

## 📂 Load Data, Models, and Results

In [ ]:
# Load scaled data
print("Loading data...")
df_scaled = pd.read_csv('clustering_scaled_standard.csv')
X = df_scaled.values

print(f"Data shape: {X.shape}")
print(f"Features: {X.shape[1]}")
print(f"Samples: {X.shape[0]:,}")

# Load original cleaned data for profiling
df_original = pd.read_csv('clustering_cleaned.csv')
print(f"\n✓ Original data loaded for cluster profiling")

In [ ]:
# Load clustering results
print("\nLoading clustering results...")

with open('all_clustering_results.pkl', 'rb') as f:
    all_results = pickle.load(f)

# Load labels
labels_df = pd.read_csv('clustering_labels.csv')

# Load comparison results
comparison_df = pd.read_csv('clustering_results.csv')

print(f"✓ Loaded results for {len(all_results)} models")
print(f"\nModels: {list(labels_df.columns)}")

## 📊 Section 1: Performance Metrics Summary

In [ ]:
# Display detailed comparison
print("="*80)
print("CLUSTERING MODELS - PERFORMANCE COMPARISON")
print("="*80)

display(comparison_df.style.background_gradient(subset=['Silhouette'], cmap='Greens')
                           .background_gradient(subset=['Calinski-Harabasz'], cmap='Blues'))

# Identify best models
print("\n" + "="*80)
print("🏆 BEST MODELS BY METRIC")
print("="*80)

best_silhouette = comparison_df.loc[comparison_df['Silhouette'].idxmax()]
print(f"\n✨ Best Silhouette Score: {best_silhouette['Model']} ({best_silhouette['Silhouette']:.4f})")

best_calinski = comparison_df.loc[comparison_df['Calinski-Harabasz'].idxmax()]
print(f"✨ Best Calinski-Harabasz: {best_calinski['Model']} ({best_calinski['Calinski-Harabasz']:.2f})")

# Davies-Bouldin (lower is better)
db_valid = comparison_df[comparison_df['Davies-Bouldin'] != float('inf')]
if len(db_valid) > 0:
    best_db = db_valid.loc[db_valid['Davies-Bouldin'].idxmin()]
    print(f"✨ Best Davies-Bouldin: {best_db['Model']} ({best_db['Davies-Bouldin']:.4f})")

## 🎨 Section 2: Cluster Visualizations (2D & 3D)

In [ ]:
# Apply PCA for 2D visualization
print("Applying PCA for 2D visualization...")
pca_2d = PCA(n_components=2, random_state=42)
X_pca_2d = pca_2d.fit_transform(X)

explained_var_2d = pca_2d.explained_variance_ratio_
print(f"✓ PCA 2D: {explained_var_2d[0]*100:.2f}% + {explained_var_2d[1]*100:.2f}% = {sum(explained_var_2d)*100:.2f}% variance explained")

# Apply PCA for 3D visualization
print("\nApplying PCA for 3D visualization...")
pca_3d = PCA(n_components=3, random_state=42)
X_pca_3d = pca_3d.fit_transform(X)

explained_var_3d = pca_3d.explained_variance_ratio_
print(f"✓ PCA 3D: {sum(explained_var_3d)*100:.2f}% variance explained")

In [ ]:
# 2D PCA Visualization for all models
model_names = list(labels_df.columns)
n_models = len(model_names)

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

for idx, model_name in enumerate(model_names):
    ax = axes[idx]
    labels = labels_df[model_name].values
    
    # Get unique clusters (exclude noise if present)
    unique_labels = np.unique(labels)
    n_clusters = len(unique_labels[unique_labels >= 0])
    
    # Plot clusters
    for label in unique_labels:
        if label == -1:
            # Noise points (DBSCAN)
            mask = labels == label
            ax.scatter(X_pca_2d[mask, 0], X_pca_2d[mask, 1], 
                      c='gray', marker='x', s=30, alpha=0.3, label='Noise')
        else:
            mask = labels == label
            ax.scatter(X_pca_2d[mask, 0], X_pca_2d[mask, 1], 
                      s=50, alpha=0.6, label=f'Cluster {label}', edgecolors='black', linewidth=0.5)
    
    ax.set_xlabel(f'PC1 ({explained_var_2d[0]*100:.1f}%)', fontweight='bold')
    ax.set_ylabel(f'PC2 ({explained_var_2d[1]*100:.1f}%)', fontweight='bold')
    ax.set_title(f'{model_name}\n({n_clusters} clusters)', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3)
    
    # Add legend if not too many clusters
    if n_clusters <= 10:
        ax.legend(loc='best', fontsize=8, framealpha=0.9)

plt.tight_layout()
plt.savefig('clusters_pca_2d.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ 2D PCA visualization saved: clusters_pca_2d.png")

In [ ]:
# 3D PCA Visualization for best model
from mpl_toolkits.mplot3d import Axes3D

best_model_name = comparison_df.iloc[0]['Model']
best_labels = labels_df[best_model_name].values

fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

unique_labels = np.unique(best_labels[best_labels >= 0])
colors = plt.cm.tab10(np.linspace(0, 1, len(unique_labels)))

for idx, label in enumerate(unique_labels):
    mask = best_labels == label
    ax.scatter(X_pca_3d[mask, 0], X_pca_3d[mask, 1], X_pca_3d[mask, 2],
              c=[colors[idx]], s=50, alpha=0.6, label=f'Cluster {label}', 
              edgecolors='black', linewidth=0.5)

# Handle noise points if present
if -1 in best_labels:
    mask = best_labels == -1
    ax.scatter(X_pca_3d[mask, 0], X_pca_3d[mask, 1], X_pca_3d[mask, 2],
              c='gray', marker='x', s=30, alpha=0.3, label='Noise')

ax.set_xlabel(f'PC1 ({explained_var_3d[0]*100:.1f}%)', fontweight='bold', fontsize=12)
ax.set_ylabel(f'PC2 ({explained_var_3d[1]*100:.1f}%)', fontweight='bold', fontsize=12)
ax.set_zlabel(f'PC3 ({explained_var_3d[2]*100:.1f}%)', fontweight='bold', fontsize=12)
ax.set_title(f'3D PCA Visualization - {best_model_name}\n(Best Model)', 
             fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='best', fontsize=10)

plt.tight_layout()
plt.savefig('clusters_pca_3d_best.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ 3D visualization saved: clusters_pca_3d_best.png")

## 📈 Section 3: Silhouette Analysis

In [ ]:
# Detailed silhouette analysis for best model
print("="*80)
print(f"SILHOUETTE ANALYSIS - {best_model_name}")
print("="*80)

# Calculate silhouette scores for each sample
mask = best_labels >= 0  # Exclude noise
sample_silhouette_values = silhouette_samples(X[mask], best_labels[mask])

# Create silhouette plot
fig, ax = plt.subplots(figsize=(12, 8))

y_lower = 10
unique_labels = np.unique(best_labels[best_labels >= 0])
colors = plt.cm.tab10(np.linspace(0, 1, len(unique_labels)))

for idx, label in enumerate(unique_labels):
    # Get silhouette scores for this cluster
    cluster_mask = best_labels[mask] == label
    cluster_silhouette_values = sample_silhouette_values[cluster_mask]
    cluster_silhouette_values.sort()
    
    size_cluster = cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster
    
    ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette_values,
                     facecolor=colors[idx], edgecolor=colors[idx], alpha=0.7)
    
    # Label the silhouette plots with their cluster numbers at the middle
    ax.text(-0.05, y_lower + 0.5 * size_cluster, f'Cluster {label}', 
            fontsize=10, fontweight='bold')
    
    y_lower = y_upper + 10

# Overall average silhouette score
avg_silhouette = silhouette_score(X[mask], best_labels[mask])
ax.axvline(x=avg_silhouette, color='red', linestyle='--', linewidth=2, 
           label=f'Average Silhouette: {avg_silhouette:.3f}')

ax.set_xlabel('Silhouette Coefficient', fontweight='bold', fontsize=12)
ax.set_ylabel('Cluster', fontweight='bold', fontsize=12)
ax.set_title(f'Silhouette Analysis - {best_model_name}', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.set_yticks([])
ax.grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('silhouette_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Silhouette analysis saved: silhouette_analysis.png")
print(f"\nAverage Silhouette Score: {avg_silhouette:.4f}")

# Per-cluster statistics
print("\nPer-Cluster Silhouette Scores:")
for label in unique_labels:
    cluster_mask = best_labels[mask] == label
    cluster_silhouette = sample_silhouette_values[cluster_mask].mean()
    cluster_size = cluster_mask.sum()
    print(f"  Cluster {label}: {cluster_silhouette:.4f} ({cluster_size:,} samples)")

## 📦 Section 4: Cluster Size Distribution

In [ ]:
# Cluster size distribution for all models
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
axes = axes.flatten()

for idx, model_name in enumerate(model_names):
    ax = axes[idx]
    labels = labels_df[model_name].values
    
    # Count samples per cluster
    unique, counts = np.unique(labels[labels >= 0], return_counts=True)
    
    # Create bar plot
    bars = ax.bar([f'C{l}' for l in unique], counts, alpha=0.7, edgecolor='black')
    
    # Color bars
    colors = plt.cm.tab10(np.linspace(0, 1, len(unique)))
    for bar, color in zip(bars, colors):
        bar.set_facecolor(color)
    
    # Add percentage labels
    total = counts.sum()
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{count:,}\n({count/total*100:.1f}%)',
               ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # Handle noise
    n_noise = np.sum(labels == -1)
    if n_noise > 0:
        ax.text(0.02, 0.98, f'Noise: {n_noise:,} ({n_noise/len(labels)*100:.1f}%)',
               transform=ax.transAxes, va='top', fontsize=9,
               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    ax.set_xlabel('Cluster', fontweight='bold')
    ax.set_ylabel('Number of Samples', fontweight='bold')
    ax.set_title(f'{model_name}', fontsize=12, fontweight='bold')
    ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('cluster_size_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Cluster size distribution saved: cluster_size_distribution.png")

## 🔍 Section 5: Cluster Profiling (Best Model)

In [ ]:
# Create cluster profiles using original data
print("="*80)
print(f"CLUSTER PROFILING - {best_model_name}")
print("="*80)

# Add cluster labels to original data
df_profiling = df_original.copy()
df_profiling['Cluster'] = best_labels

# Remove noise points if any
df_profiling_clean = df_profiling[df_profiling['Cluster'] >= 0]

# Calculate statistics per cluster
cluster_profiles = df_profiling_clean.groupby('Cluster').agg(['mean', 'std', 'min', 'max'])

print("\nCluster Profiles (Mean values):")
display(cluster_profiles.xs('mean', level=1, axis=1).style.background_gradient(cmap='YlOrRd'))

# Save profiles
cluster_profiles.to_csv('cluster_profiles.csv')
print("\n✓ Cluster profiles saved: cluster_profiles.csv")

In [ ]:
# Visualize cluster profiles with radar chart
from math import pi

# Get mean values for each cluster
cluster_means = df_profiling_clean.groupby('Cluster').mean()
features = cluster_means.columns.tolist()
n_features = len(features)

# Normalize to 0-1 for radar chart
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cluster_means_scaled = pd.DataFrame(
    scaler.fit_transform(cluster_means),
    columns=features,
    index=cluster_means.index
)

# Create radar chart for each cluster
n_clusters = len(cluster_means_scaled)
angles = [n / float(n_features) * 2 * pi for n in range(n_features)]
angles += angles[:1]

fig, axes = plt.subplots(1, min(n_clusters, 3), figsize=(18, 6), 
                        subplot_kw=dict(projection='polar'))

if n_clusters == 1:
    axes = [axes]
elif n_clusters == 2:
    axes = axes
else:
    axes = axes[:min(n_clusters, 3)]

for idx, (cluster_id, ax) in enumerate(zip(cluster_means_scaled.index[:3], axes)):
    values = cluster_means_scaled.loc[cluster_id].values.tolist()
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=f'Cluster {cluster_id}')
    ax.fill(angles, values, alpha=0.25)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(features, size=8)
    ax.set_ylim(0, 1)
    ax.set_title(f'Cluster {cluster_id} Profile', size=12, fontweight='bold', pad=20)
    ax.grid(True)

plt.tight_layout()
plt.savefig('cluster_profiles_radar.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Radar chart saved: cluster_profiles_radar.png")

## 🎯 Section 6: Feature Importance for Clustering

In [ ]:
# Calculate feature variance between clusters
print("="*80)
print("FEATURE IMPORTANCE FOR CLUSTERING")
print("="*80)

# Calculate variance between cluster means
cluster_means = df_profiling_clean.groupby('Cluster').mean()
feature_variance = cluster_means.var(axis=0).sort_values(ascending=False)

print("\nFeature Variance Between Clusters (Higher = More Important):")
print(feature_variance)

# Visualize
plt.figure(figsize=(12, 6))
plt.barh(range(len(feature_variance)), feature_variance.values, alpha=0.7, color='steelblue')
plt.yticks(range(len(feature_variance)), feature_variance.index)
plt.xlabel('Variance Between Cluster Means', fontweight='bold')
plt.ylabel('Feature', fontweight='bold')
plt.title('Feature Importance for Clustering', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3, axis='x')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Feature importance saved: feature_importance.png")

## 💼 Section 7: Business Insights & Customer Segments

In [ ]:
# Generate business insights
print("="*80)
print("BUSINESS INSIGHTS & CUSTOMER SEGMENTATION")
print("="*80)

unique_clusters = np.unique(best_labels[best_labels >= 0])

# For each cluster, identify key characteristics
for cluster_id in unique_clusters:
    print(f"\n{'='*60}")
    print(f"CLUSTER {cluster_id} - CUSTOMER SEGMENT")
    print(f"{'='*60}")
    
    cluster_data = df_profiling_clean[df_profiling_clean['Cluster'] == cluster_id]
    cluster_size = len(cluster_data)
    cluster_pct = cluster_size / len(df_profiling_clean) * 100
    
    print(f"\n📊 Size: {cluster_size:,} customers ({cluster_pct:.1f}% of total)")
    
    # Find distinguishing features (highest deviation from overall mean)
    overall_mean = df_profiling_clean.drop('Cluster', axis=1).mean()
    cluster_mean = cluster_data.drop('Cluster', axis=1).mean()
    deviations = ((cluster_mean - overall_mean) / overall_mean * 100).abs().sort_values(ascending=False)
    
    print(f"\n🔍 Top 5 Distinguishing Features:")
    for feature in deviations.head(5).index:
        cluster_val = cluster_mean[feature]
        overall_val = overall_mean[feature]
        deviation = deviations[feature]
        direction = "higher" if cluster_val > overall_val else "lower"
        print(f"   • {feature}: {cluster_val:.2f} ({deviation:.1f}% {direction} than average)")
    
    # Statistical summary
    print(f"\n📈 Statistical Summary:")
    summary = cluster_data.drop('Cluster', axis=1).describe().loc[['mean', 'std']]
    print(summary.T.to_string())

print("\n" + "="*80)
print("✅ BUSINESS INSIGHTS COMPLETE")
print("="*80)

## 📝 Section 8: Final Summary Report

In [ ]:
# Generate comprehensive summary report
print("="*80)
print("📊 CLUSTERING EVALUATION - FINAL SUMMARY REPORT")
print("="*80)

print("\n1️⃣ MODELS EVALUATED")
print(f"   Total models: {len(all_results)}")
print(f"   Models: {', '.join(model_names)}")

print("\n2️⃣ BEST PERFORMING MODEL")
print(f"   🏆 Model: {best_model_name}")
print(f"   📊 Silhouette Score: {best_silhouette['Silhouette']:.4f}")
print(f"   📊 Calinski-Harabasz: {best_silhouette['Calinski-Harabasz']:.2f}")
print(f"   📊 Number of Clusters: {best_silhouette['N_Clusters']}")

print("\n3️⃣ CLUSTER CHARACTERISTICS")
for cluster_id in unique_clusters:
    cluster_size = len(df_profiling_clean[df_profiling_clean['Cluster'] == cluster_id])
    cluster_pct = cluster_size / len(df_profiling_clean) * 100
    print(f"   • Cluster {cluster_id}: {cluster_size:,} customers ({cluster_pct:.1f}%)")

print("\n4️⃣ DATA QUALITY")
print(f"   • Total samples: {len(X):,}")
print(f"   • Features: {X.shape[1]}")
print(f"   • Noise points: {np.sum(best_labels == -1)} ({np.sum(best_labels == -1)/len(best_labels)*100:.2f}%)")

print("\n5️⃣ OUTPUT FILES CREATED")
print("   ✅ clustering_results.csv - Model comparison")
print("   ✅ cluster_profiles.csv - Cluster characteristics")
print("   ✅ clusters_pca_2d.png - 2D visualizations")
print("   ✅ clusters_pca_3d_best.png - 3D visualization")
print("   ✅ silhouette_analysis.png - Silhouette plot")
print("   ✅ cluster_size_distribution.png - Size distribution")
print("   ✅ cluster_profiles_radar.png - Radar charts")
print("   ✅ feature_importance.png - Feature analysis")

print("\n6️⃣ RECOMMENDATIONS")
print(f"   📌 Use {best_model_name} for production")
print(f"   📌 Focus on top {min(5, len(feature_variance))} distinguishing features")
print(f"   📌 Develop targeted strategies for each of {len(unique_clusters)} customer segments")
print(f"   📌 Monitor cluster stability over time")

print("\n" + "="*80)
print("✅ CLUSTERING EVALUATION COMPLETE!")
print("="*80)

## 💾 Export Final Results

In [ ]:
# Export data with cluster assignments
print("Exporting final results...")

# Add best model's clusters to original data
df_final = df_original.copy()
df_final['Cluster'] = best_labels
df_final['Model'] = best_model_name

# Save
df_final.to_csv('customers_with_clusters.csv', index=False)
print("✓ Final dataset with clusters saved: customers_with_clusters.csv")

# Save summary statistics
summary_stats = {
    'Best_Model': best_model_name,
    'N_Clusters': len(unique_clusters),
    'Silhouette_Score': best_silhouette['Silhouette'],
    'Calinski_Harabasz': best_silhouette['Calinski-Harabasz'],
    'Davies_Bouldin': best_silhouette['Davies-Bouldin'],
    'Total_Samples': len(X),
    'Noise_Points': int(np.sum(best_labels == -1))
}

with open('clustering_summary.txt', 'w') as f:
    f.write("CLUSTERING EVALUATION SUMMARY\n")
    f.write("=" * 50 + "\n\n")
    for key, value in summary_stats.items():
        f.write(f"{key}: {value}\n")

print("✓ Summary statistics saved: clustering_summary.txt")
print("\n🎉 All results exported successfully!")